In [1]:
import json
import pandas as pd

### Load Data

In [2]:
data = pd.read_csv('../student-por.csv', delimiter=';')
data.head(5)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [3]:
drop_col = ['school','sex','age','Mjob', 'Fjob','reason','guardian']
data = data.drop(columns=drop_col)
data.head(5)

,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,U,GT3,A,4,4,2,2,0,yes,no,...,4,3,4,1,1,3,4,0,11,11
1,U,GT3,T,1,1,1,2,0,no,yes,...,5,3,3,1,1,3,2,9,11,11
2,U,LE3,T,1,1,1,2,0,yes,no,...,4,3,2,2,3,3,6,12,13,12
3,U,GT3,T,4,2,1,3,0,no,yes,...,3,2,2,1,1,5,0,14,14,14
4,U,GT3,T,3,3,1,2,0,no,yes,...,4,3,2,1,2,5,0,11,13,13


### Constructing the DAG

In [1]:
from causalvis import DAG

In [5]:
import numpy as np

struct_data = data.copy()
non_numeric_columns = list(struct_data.select_dtypes(exclude=[np.number]).columns)

print(non_numeric_columns)

['address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']


In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in non_numeric_columns:
    struct_data[col] = le.fit_transform(struct_data[col])

struct_data.head(5)

,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,1,0,0,4,4,2,2,0,1,0,...,4,3,4,1,1,3,4,0,11,11
1,1,0,1,1,1,1,2,0,0,1,...,5,3,3,1,1,3,2,9,11,11
2,1,1,1,1,1,1,2,0,1,0,...,4,3,2,2,3,3,6,12,13,12
3,1,0,1,4,2,1,3,0,0,1,...,3,2,2,1,1,5,0,14,14,14
4,1,0,1,3,3,1,2,0,0,1,...,4,3,2,1,2,5,0,11,13,13


### Editing the DAG

You can pass all the attributes in the dataset to the DAG widget and construct your own DAG.
If you would like to skip this step, you can load the example from the saved DAG file in the next cell.

In [7]:
DAG(data=struct_data)

DAG(component='DAG', props={'attributes': ['absences', 'activities', 'address', 'Dalc', 'failures', 'famrel', …

### Identifying control variables

Once we are satisfied with the DAG, we can set the treatment and outcome variables. In this case, we are interested in `G1` as the outcome variable, while `absences` is the treatment variable of interest. After selecting these variables, we see that other attributes in the DAG are automatically colored to reflect their relationship to the treatment and outcome.

From the downloads button, we can get a list of the `confounds` and `prognostic` factors to control for.

In [8]:
with open('../student_por_DAG.json', 'r') as d:
    graph = json.load(d)

In [9]:
DAG(graph=graph)

DAG(component='DAG', props={'attributes': None, 'graph': {'nodes': [{'x': 500, 'y': 250, 'id': 1658176204175, …

In [10]:
confounds = ["failures", "address", "paid", "internet", "Pstatus"]
prognostics = ["higher", "schoolsup", "studytime"]

# Cohort Construction and Refinement

In [11]:
from causalvis import CohortEvaluator

In [12]:
data.shape

(649, 26)

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = pd.Series(le.fit_transform(data['G1']))
y.mean()

8.403697996918336

In [14]:
a = pd.Series(le.fit_transform(data['absences']))
a.mean()

3.596302003081664

In [15]:
def toBinary(x):
    if x < 4: return 0
    else: return 1
a = a.apply(toBinary)

In [16]:
confounders = confounds + prognostics

X = data[confounders]
X.dtypes

failures      int64
address      object
paid         object
internet     object
Pstatus      object
higher       object
schoolsup    object
studytime     int64
dtype: object

In [17]:
X = pd.get_dummies(X, prefix_sep='=', drop_first=True)
X.shape

(649, 8)

In [18]:
from sklearn.linear_model import LogisticRegression
from causallib.estimation import IPW

lr = LogisticRegression(solver='lbfgs', max_iter=1000)
ipw = IPW(lr)

In [19]:
ipw.fit(X, a)

IPW(clip_max=None, clip_min=None, use_stabilized=False, verbose=False,
    learner=LogisticRegression(max_iter=1000))

In [20]:
propMatrix = ipw.compute_propensity_matrix(X, a).to_dict(orient="records")

In [21]:
def formatData(X, a, y, propMatrix):
    unadjustedData = []

    confounds = X.to_dict(orient="records")
    a.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)

    for i in range(len(confounds)):
        newDataInstance = confounds[i]
        newDataInstance['treatment'] = a[i]
        # newDataInstance['outcome'] = y[i]
        newDataInstance['propensity'] = propMatrix[i]

        unadjustedData.append(newDataInstance)
        
    return unadjustedData

In [22]:
ceval = CohortEvaluator(unadjustedCohort=formatData(X, a, y, propMatrix))
ceval

CohortEvaluator(component='CohortEvaluator', props={'unadjustedCohort': [{'failures': 0, 'studytime': 2, 'addr…

In [23]:
# ceval.selection

# IPW

Since the population is well balanced, we can go ahead and estimate the treatment effect. This can be done quite easily in causallib, using the `estimate_population_outcome` function. From the result, we can see that having < 4 absences a year causes a slightly higher score for `G1`.

In [24]:
ipw.estimate_population_outcome(X, a, y)

0    8.662879
1    8.040071
dtype: float64

# Matching

In [25]:
from causallib.estimation import IPW, Matching
from causallib.preprocessing.transformers import PropensityTransformer
from sklearn.linear_model import LogisticRegression
import pandas as pd


def learner(): return LogisticRegression(solver="liblinear",
                                         max_iter=5000,
                                         class_weight="balanced")

In [26]:
try:
    from causallib.contrib.faissknn import FaissNearestNeighbors
    knn_backend = FaissNearestNeighbors
except ImportError:
    knn_backend = "sklearn"

In [40]:
from causallib.preprocessing.transformers import MatchingTransformer
from causallib.evaluation.weight_evaluator import calculate_covariate_balance
caliper = np.logspace(-5.5, -1, 10)[5]
covbal = []
n_neighbors = 3

# def match_then_ipw_weight(caliper):
mt = MatchingTransformer(
    propensity_transform=PropensityTransformer(
        include_covariates=False, learner=learner()),
    caliper=caliper,
    n_neighbors=n_neighbors)
mt.fit(X, a, y)
Xm, am, ym = mt.transform(X, a, y)

ipw = IPW(learner=learner())
ipw.fit(Xm, am)
propMatrixm = ipw.compute_propensity_matrix(Xm, am).to_dict(orient="records")

In [41]:
Xm.reset_index(drop=True, inplace=True)
am.reset_index(drop=True, inplace=True)
ym.reset_index(drop=True, inplace=True)
Xm.shape

(587, 8)

In [42]:
cevalm = CohortEvaluator(adjustedCohort=formatData(Xm, am, ym, propMatrixm), unadjustedCohort=formatData(X, a, y, propMatrix))
cevalm

CohortEvaluator(component='CohortEvaluator', props={'unadjustedCohort': [{'failures': 0, 'studytime': 2, 'addr…

In [43]:
propensity_transform = PropensityTransformer(
    include_covariates=False, learner=learner())
matcher = Matching(propensity_transform=propensity_transform,
                   with_replacement=True, n_neighbors=1, knn_backend=knn_backend)
matcher.fit(Xm, am, ym)
matches = matcher._execute_matching(Xm, am)

In [44]:
matcher.estimate_population_outcome(Xm, am)

0    8.453152
1    8.967632
dtype: float64

In [45]:
ind_outcomes = matcher.estimate_individual_outcome(Xm, am).to_dict(orient="records")

In [46]:
I_treatment = []
I_control = []

for i in ind_outcomes:
    i_treatment = i[1]
    i_control = i[0]
    
    I_treatment.append(i_treatment)
    I_control.append(i_control)

In [47]:
ind_effects = []

for i in range(len(I_treatment)):
    treatment_val = I_treatment[i]
    control_val = I_control[i]
    
    effect = treatment_val - control_val
    
    ind_effects.append(effect)

In [48]:
data = []
Xm_dict = Xm.to_dict(orient="records")

for i in range(len(Xm_dict)):
    newDataInstance = Xm_dict[i]
    newDataInstance['treatment'] = int(am[i])
    newDataInstance['outcome'] = int(ym[i])
    newDataInstance['effect'] = int(ind_effects[i])
    data.append(newDataInstance)

In [49]:
from causalvis import TreatmentEffectEvaluator

TreatmentEffectEvaluator(data=data)

TreatmentEffectEvaluator(component='TreatmentEffectEvaluator', props={'data': [{'failures': 0, 'studytime': 2,…

In [50]:
import json
with open('../../public/te_test_3.json', 'w') as f:
    json.dump(data, f)